# VGG MODEL

In [ ]:
import torch
import torchvision
from torch import optim
from torchvision import transforms , datasets
from torch.utils.data import DataLoader
import torchvision.transforms as transforms
import torch.nn as nn
import torch.nn.functional as F
# The following is to downlaod and use pretrained models from the model-zoo
import torchvision.models as models
from tqdm import tqdm
import numpy as np
import os
from PIL import Image

In [ ]:
batch_size=32
normalize = transforms.Normalize(
   mean=[0.485, 0.456, 0.406],
   std=[0.229, 0.224, 0.225]
)
trans = transforms.Compose([transforms.RandomCrop(32, 4), transforms.ToTensor(),normalize])

trainset = torchvision.datasets.CIFAR10(root='.', train=True, download=True, transform=trans)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size, shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='.', train=False, download=True, transform=trans)
testloader = torch.utils.data.DataLoader(testset, batch_size=2*batch_size, shuffle=False, num_workers=2)

Files already downloaded and verified
Files already downloaded and verified


In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(vgg19_bn.parameters(), lr=0.001)
# input_lastLayer = vgg19_bn.classifier[1].in_features
# vgg19_bn.classifier[1] = nn.Linear(input_lastLayer,10)
# vgg19_bn = vgg19_bn.to(device)

# VGG11 with Group Normalisation
creating custom vgg models as pretrained models couldnot be loaded due to pytorch cuda out of memory error.

In [ ]:
# -*- coding: utf-8 -*-
"""Untitled7.ipynb

Automatically generated by Colaboratory.

Original file is located at
    https://colab.research.google.com/drive/1wO-BWgR8eCOKwaw2IIG5CJ35JHpBkVyg
"""


# from vgg import vgg11_bn

import torch
import torchvision
from torch import optim
from torchvision import transforms , datasets
from torch.utils.data import DataLoader
import torchvision.transforms as transforms
import torch.nn as nn
import torch.nn.functional as F
# The following is to downlaod and use pretrained models from the model-zoo
import torchvision.models as models
from tqdm import tqdm
import numpy as np
import os
from PIL import Image

'''
Modified from https://github.com/pytorch/vision.git
'''
import math

import torch.nn as nn
import torch.nn.init as init

__all__ = [
    'VGG', 'vgg11', 'vgg11_bn', 'vgg13', 'vgg13_bn', 'vgg16', 'vgg16_bn',
    'vgg19_bn', 'vgg19',
]


class VGG(nn.Module):
    '''
    VGG model
    '''
    def __init__(self, features):
        super(VGG, self).__init__()
        self.features = features
        self.classifier = nn.Sequential(
            nn.Dropout(),
            nn.Linear(512, 512),
            nn.ReLU(True),
            nn.Dropout(),
            nn.Linear(512, 512),
            nn.ReLU(True),
            nn.Linear(512, 10),
        )
         # Initialize weights
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                n = m.kernel_size[0] * m.kernel_size[1] * m.out_channels
                m.weight.data.normal_(0, math.sqrt(2. / n))
                m.bias.data.zero_()


    def forward(self, x):
        x = self.features(x)
        x = x.view(x.size(0), -1)
        x = self.classifier(x)
        return x


def make_layers(cfg, batch_norm=False):
    layers = []
    in_channels = 3
    for v in cfg:
        if v == 'M':
            layers += [nn.MaxPool2d(kernel_size=2, stride=2)]
        else:
            conv2d = nn.Conv2d(in_channels, v, kernel_size=3, padding=1)
            if batch_norm:
                layers += [conv2d, nn.GroupNorm(32,v), nn.ReLU(inplace=True)]
            else:
                layers += [conv2d, nn.ReLU(inplace=True)]
            in_channels = v
    return nn.Sequential(*layers)


cfg = {
    'A': [64, 'M', 128, 'M', 256, 256, 'M', 512, 512, 'M', 512, 512, 'M'],
    'B': [64, 64, 'M', 128, 128, 'M', 256, 256, 'M', 512, 512, 'M', 512, 512, 'M'],
    'D': [64, 64, 'M', 128, 128, 'M', 256, 256, 256, 'M', 512, 512, 512, 'M', 512, 512, 512, 'M'],
    'E': [64, 64, 'M', 128, 128, 'M', 256, 256, 256, 256, 'M', 512, 512, 512, 512, 'M',
          512, 512, 512, 512, 'M'],
}


def vgg11():
    """VGG 11-layer model (configuration "A")"""
    return VGG(make_layers(cfg['A']))


def vgg11_bn():
    """VGG 11-layer model (configuration "A") with batch normalization"""
    return VGG(make_layers(cfg['A'], batch_norm=True))


def vgg13():
    """VGG 13-layer model (configuration "B")"""
    return VGG(make_layers(cfg['B']))


def vgg13_bn():
    """VGG 13-layer model (configuration "B") with batch normalization"""
    return VGG(make_layers(cfg['B'], batch_norm=True))


def vgg16():
    """VGG 16-layer model (configuration "D")"""
    return VGG(make_layers(cfg['D']))


def vgg16_bn():
    """VGG 16-layer model (configuration "D") with batch normalization"""
    return VGG(make_layers(cfg['D'], batch_norm=True))


def vgg19():
    """VGG 19-layer model (configuration "E")"""
    return VGG(make_layers(cfg['E']))


def vgg19_bn():
    """VGG 19-layer model (configuration 'E') with batch normalization"""
    return VGG(make_layers(cfg['E'], batch_norm=True))

model=vgg11_bn()

In [ ]:
model

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): GroupNorm(32, 64, eps=1e-05, affine=True)
    (2): ReLU(inplace=True)
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (4): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (5): GroupNorm(32, 128, eps=1e-05, affine=True)
    (6): ReLU(inplace=True)
    (7): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (8): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): GroupNorm(32, 256, eps=1e-05, affine=True)
    (10): ReLU(inplace=True)
    (11): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (12): GroupNorm(32, 256, eps=1e-05, affine=True)
    (13): ReLU(inplace=True)
    (14): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (15): Conv2d(256, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (16): G

In [ ]:
if torch.cuda.is_available():
    model.cuda()
# # Freeze the layers
# for param in model.parameters():
#     param.requires_grad = False

from torchsummary import summary
summary(model, (3, 32, 32))
print(model)

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), 0.001)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

#TRAINING THE NETWORK using GN
from tqdm import tqdm
model = model.to(device)
for epoch in range(10):  # loop over the dataset multiple times
    running_loss = 0.0
    pbar = tqdm(trainloader)
    i = 0
    for data in pbar:
        # get the inputs
        inputs, labels = data
        inputs = inputs.to(device)
        labels = labels.to(device)

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()

        pbar.set_description("Processing epoch {:d} minibatch {:d} train loss {:.3f}".format(epoch,\
                                                            i+1, running_loss/(i+1)))
        i += 1

print('Finished Training')

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 64, 32, 32]           1,792
         GroupNorm-2           [-1, 64, 32, 32]             128
              ReLU-3           [-1, 64, 32, 32]               0
         MaxPool2d-4           [-1, 64, 16, 16]               0
            Conv2d-5          [-1, 128, 16, 16]          73,856
         GroupNorm-6          [-1, 128, 16, 16]             256
              ReLU-7          [-1, 128, 16, 16]               0
         MaxPool2d-8            [-1, 128, 8, 8]               0
            Conv2d-9            [-1, 256, 8, 8]         295,168
        GroupNorm-10            [-1, 256, 8, 8]             512
             ReLU-11            [-1, 256, 8, 8]               0
           Conv2d-12            [-1, 256, 8, 8]         590,080
        GroupNorm-13            [-1, 256, 8, 8]             512
             ReLU-14            [-1, 25

Processing epoch 0 minibatch 196 train loss 2.318: 100%|██████████| 196/196 [00:22<00:00,  8.58it/s]
Processing epoch 1 minibatch 196 train loss 2.310: 100%|██████████| 196/196 [00:22<00:00,  8.54it/s]
Processing epoch 2 minibatch 196 train loss 2.305: 100%|██████████| 196/196 [00:20<00:00,  9.69it/s]
Processing epoch 3 minibatch 196 train loss 2.301: 100%|██████████| 196/196 [00:21<00:00,  9.10it/s]
Processing epoch 4 minibatch 196 train loss 2.298: 100%|██████████| 196/196 [00:21<00:00,  9.28it/s]
Processing epoch 5 minibatch 196 train loss 2.293: 100%|██████████| 196/196 [00:20<00:00,  9.74it/s]
Processing epoch 6 minibatch 196 train loss 2.287: 100%|██████████| 196/196 [00:21<00:00,  9.00it/s]
Processing epoch 7 minibatch 196 train loss 2.280: 100%|██████████| 196/196 [00:20<00:00,  9.73it/s]
Processing epoch 8 minibatch 196 train loss 2.270: 100%|██████████| 196/196 [00:20<00:00,  9.39it/s]
Processing epoch 9 minibatch 196 train loss 2.255: 100%|██████████| 196/196 [00:20<00:00,  

Finished Training


# Testing model accuracy on normal images

In [ ]:
from tqdm import tqdm
correct = 0
total = 0
i = 0
model.eval()
with torch.no_grad():
    pbar = tqdm(testloader)
    for data in pbar:
        images, labels = data
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
        pbar.set_description("minibatch {:d} test accuracy {:4.2f}%".format(i+1,\
                                                            100.0*correct/total))
        i += 1

print('Accuracy of the network on the 10000 test images: %4.2f %%' % (100.0 * correct / total))


minibatch 20 test accuracy 22.77%: 100%|██████████| 20/20 [00:03<00:00,  5.63it/s]

Accuracy of the network on the 10000 test images: 22.77 %


# Loading corrupted images from drive

In [ ]:
import os
from PIL import Image
path = '/content/drive/MyDrive/CV_final_project_files/'
filenames = [filename for filename in os.listdir(path) if filename.endswith('.jpg')]
filenames.sort()

images = []
for filename in filenames:
    img = Image.open(os.path.join(path, filename))

    images.append(trans(img))

labels=[]
for tensor in testloader:
  for label in tensor[1]:
    labels.append(label.item())

currtestset = torch.utils.data.TensorDataset(torch.stack(images), torch.tensor(labels))
currtestloader = torch.utils.data.DataLoader(currtestset, batch_size=2*batch_size, shuffle=False, num_workers=2)

# testing model accuracy on corrupted images

In [ ]:
from tqdm import tqdm
correct = 0
total = 0
i = 0
model.eval()
with torch.no_grad():
    pbar = tqdm(currtestloader)
    for data in pbar:
        images, labels = data
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
        pbar.set_description("minibatch {:d} test accuracy {:4.2f}%".format(i+1,\
                                                            100.0*correct/total))
        i += 1
print('Accuracy of the network on the 10000 test images: %4.2f %%' % (100.0 * correct / total))

minibatch 47 test accuracy 20.99%: 100%|██████████| 47/47 [00:07<00:00,  6.33it/s]

Accuracy of the network on the 10000 test images: 20.99 %


# VGG11 with Batch Normalisation

In [ ]:
# -*- coding: utf-8 -*-
"""Untitled7.ipynb

Automatically generated by Colaboratory.

Original file is located at
    https://colab.research.google.com/drive/1wO-BWgR8eCOKwaw2IIG5CJ35JHpBkVyg
"""


# from vgg import vgg11_bn

import torch
import torchvision
from torch import optim
from torchvision import transforms , datasets
from torch.utils.data import DataLoader
import torchvision.transforms as transforms
import torch.nn as nn
import torch.nn.functional as F
# The following is to downlaod and use pretrained models from the model-zoo
import torchvision.models as models
from tqdm import tqdm
import numpy as np
import os
from PIL import Image

'''
Modified from https://github.com/pytorch/vision.git
'''
import math

import torch.nn as nn
import torch.nn.init as init

__all__ = [
    'VGG_bn', 'vgg11', 'vgg11_bn', 'vgg13', 'vgg13_bn', 'vgg16', 'vgg16_bn',
    'vgg19_bn', 'vgg19',
]


class VGG_bn(nn.Module):
    '''
    VGG model
    '''
    def __init__(self, features):
        super(VGG_bn, self).__init__()
        self.features = features
        self.classifier = nn.Sequential(
            nn.Dropout(),
            nn.Linear(512, 512),
            nn.ReLU(True),
            nn.Dropout(),
            nn.Linear(512, 512),
            nn.ReLU(True),
            nn.Linear(512, 10),
        )
         # Initialize weights
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                n = m.kernel_size[0] * m.kernel_size[1] * m.out_channels
                m.weight.data.normal_(0, math.sqrt(2. / n))
                m.bias.data.zero_()


    def forward(self, x):
        x = self.features(x)
        x = x.view(x.size(0), -1)
        x = self.classifier(x)
        return x


def make_layers(cfg, batch_norm=False):
    layers = []
    in_channels = 3
    for v in cfg:
        if v == 'M':
            layers += [nn.MaxPool2d(kernel_size=2, stride=2)]
        else:
            conv2d = nn.Conv2d(in_channels, v, kernel_size=3, padding=1)
            if batch_norm:
                layers += [conv2d, nn.BatchNorm2d(v), nn.ReLU(inplace=True)]
            else:
                layers += [conv2d, nn.ReLU(inplace=True)]
            in_channels = v
    return nn.Sequential(*layers)


cfg = {
    'A': [64, 'M', 128, 'M', 256, 256, 'M', 512, 512, 'M', 512, 512, 'M'],
    'B': [64, 64, 'M', 128, 128, 'M', 256, 256, 'M', 512, 512, 'M', 512, 512, 'M'],
    'D': [64, 64, 'M', 128, 128, 'M', 256, 256, 256, 'M', 512, 512, 512, 'M', 512, 512, 512, 'M'],
    'E': [64, 64, 'M', 128, 128, 'M', 256, 256, 256, 256, 'M', 512, 512, 512, 512, 'M',
          512, 512, 512, 512, 'M'],
}


def vgg11():
    """VGG 11-layer model (configuration "A")"""
    return VGG_bn(make_layers(cfg['A']))


def vgg11_bn():
    """VGG 11-layer model (configuration "A") with batch normalization"""
    return VGG_bn(make_layers(cfg['A'], batch_norm=True))


def vgg13():
    """VGG 13-layer model (configuration "B")"""
    return VGG_bn(make_layers(cfg['B']))


def vgg13_bn():
    """VGG 13-layer model (configuration "B") with batch normalization"""
    return VGG(make_layers(cfg['B'], batch_norm=True))


def vgg16():
    """VGG 16-layer model (configuration "D")"""
    return VGG_bn(make_layers(cfg['D']))


def vgg16_bn():
    """VGG 16-layer model (configuration "D") with batch normalization"""
    return VGG_bn(make_layers(cfg['D'], batch_norm=True))


def vgg19():
    """VGG 19-layer model (configuration "E")"""
    return VGG_bn(make_layers(cfg['E']))


def vgg19_bn():
    """VGG 19-layer model (configuration 'E') with batch normalization"""
    return VGG_bn(make_layers(cfg['E'], batch_norm=True))


In [ ]:
model_bn=vgg11_bn()

In [ ]:
model_bn

VGG_bn(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (4): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (5): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (6): ReLU(inplace=True)
    (7): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (8): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (10): ReLU(inplace=True)
    (11): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (12): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (13): ReLU(inplace=True)
    (14): MaxPool2d

In [ ]:
if torch.cuda.is_available():
    model_bn.cuda()
# # Freeze the layers
# for param in model.parameters():
#     param.requires_grad = False

from torchsummary import summary
summary(model_bn, (3, 32, 32))

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), 0.001)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

#TRAINING THE NETWORK using GN
from tqdm import tqdm
model_bn = model_bn.to(device)
for epoch in range(10):  # loop over the dataset multiple times
    running_loss = 0.0
    pbar = tqdm(trainloader)
    i = 0
    for data in pbar:
        # get the inputs
        inputs, labels = data
        inputs = inputs.to(device)
        labels = labels.to(device)

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = model_bn(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()

        pbar.set_description("Processing epoch {:d} minibatch {:d} train loss {:.3f}".format(epoch,\
                                                            i+1, running_loss/(i+1)))
        i += 1

print('Finished Training')

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 64, 32, 32]           1,792
       BatchNorm2d-2           [-1, 64, 32, 32]             128
              ReLU-3           [-1, 64, 32, 32]               0
         MaxPool2d-4           [-1, 64, 16, 16]               0
            Conv2d-5          [-1, 128, 16, 16]          73,856
       BatchNorm2d-6          [-1, 128, 16, 16]             256
              ReLU-7          [-1, 128, 16, 16]               0
         MaxPool2d-8            [-1, 128, 8, 8]               0
            Conv2d-9            [-1, 256, 8, 8]         295,168
      BatchNorm2d-10            [-1, 256, 8, 8]             512
             ReLU-11            [-1, 256, 8, 8]               0
           Conv2d-12            [-1, 256, 8, 8]         590,080
      BatchNorm2d-13            [-1, 256, 8, 8]             512
             ReLU-14            [-1, 25

Processing epoch 0 minibatch 1563 train loss 2.312: 100%|██████████| 1563/1563 [00:38<00:00, 40.33it/s]
Processing epoch 1 minibatch 1563 train loss 2.312: 100%|██████████| 1563/1563 [00:37<00:00, 41.92it/s]
Processing epoch 2 minibatch 1563 train loss 2.312: 100%|██████████| 1563/1563 [00:37<00:00, 41.52it/s]
Processing epoch 3 minibatch 1563 train loss 2.312: 100%|██████████| 1563/1563 [00:37<00:00, 41.46it/s]
Processing epoch 4 minibatch 1563 train loss 2.312: 100%|██████████| 1563/1563 [00:37<00:00, 41.16it/s]
Processing epoch 5 minibatch 1563 train loss 2.312: 100%|██████████| 1563/1563 [00:36<00:00, 42.27it/s]
Processing epoch 6 minibatch 1563 train loss 2.312: 100%|██████████| 1563/1563 [00:38<00:00, 40.59it/s]
Processing epoch 7 minibatch 1563 train loss 2.312: 100%|██████████| 1563/1563 [00:37<00:00, 41.14it/s]
Processing epoch 8 minibatch 1563 train loss 2.312: 100%|██████████| 1563/1563 [00:37<00:00, 41.92it/s]
Processing epoch 9 minibatch 1563 train loss 2.312: 100%|███████

Finished Training


In [ ]:
from tqdm import tqdm
correct = 0
total = 0
i = 0
model_bn.eval()
with torch.no_grad():
    pbar = tqdm(testloader)
    for data in pbar:
        images, labels = data
        images = images.to(device)
        labels = labels.to(device)
        outputs = model_bn(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
        pbar.set_description("minibatch {:d} test accuracy {:4.2f}%".format(i+1,\
                                                            100.0*correct/total))
        i += 1

print('Accuracy of the network on the 10000 test images: %4.2f %%' % (100.0 * correct / total))


minibatch 157 test accuracy 10.22%: 100%|██████████| 157/157 [00:08<00:00, 18.77it/s]

Accuracy of the network on the 10000 test images: 10.22 %


In [ ]:
from tqdm import tqdm
correct = 0
total = 0
i = 0
with torch.no_grad():
    pbar = tqdm(currtestloader)
    for data in pbar:
        images, labels = data
        images = images.to(device)
        labels = labels.to(device)
        outputs = model_bn(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
        pbar.set_description("minibatch {:d} test accuracy {:4.2f}%".format(i+1,\
                                                            100.0*correct/total))
        i += 1
print('Accuracy of the network on the 10000 test images: %4.2f %%' % (100.0 * correct / total))

minibatch 47 test accuracy 9.59%: 100%|██████████| 47/47 [00:00<00:00, 75.95it/s]

Accuracy of the network on the 10000 test images: 9.59 %



#         Finished

